In [1]:
# Import libraries
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from IPython.display import Markdown as md
from IPython.core.magic import register_cell_magic
from tabulate import tabulate
from sklearn.linear_model import LinearRegression
from IPython.display import Latex
import mercury as mr

# Allow for markdown code cells
@register_cell_magic
def markdown(line, cell):
    return md(cell.format(**globals()))

In [21]:
app = mr.App(title="Lab 2", description="Lab report for PHYS 2125 (15921) Spring 2025 Lab 2.")

mercury.App

In [22]:
# Request Data
name = mr.Text(value="Your Name", label="What is your name?", rows=1)
course = mr.Text(value="PHYS 2125 (15921)", label="What class are you in?", rows=1)
date = mr.Text(value="2025-01-24", label="Lab Date", rows=1)
labNum = mr.Text(value="2", label="Lab Number", rows=1)
labName = mr.Text(value="Simple Pendulum", label="Lab Name", rows=1)
file = mr.File(label="Data file upload", max_file_size="10MB")
dq1 = mr.Text(value="Your Answer to Question 1", label="What is the most challenging measurements during data collection and explain why? (4 pts)", rows=20)
dq2 = mr.Text(value="Your Answer to Question 2", label="Why do you include half of the height of the cylinder for the length of the pendulum? (3 pts)", rows=20)
dq3 = mr.Text(value="Your Answer to Question 3", label="What changes of the period of the pendulum you would expect, if you changed the mass of the pendulum? (3 pts)", rows=20)

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.File

mercury.Text

mercury.Text

mercury.Text

In [80]:
# Parse Data

dataColumnNames = ["l (meters)", "h (meters)","L (meters)","5T₁ (seconds)","5T₂ (seconds)","T₁ (seconds)","T₂ (seconds)","T (seconds)","T² (seconds²)","g (meters/seconds²)"];

if file.filepath is not None:
    df = pd.read_csv(file.filepath, names=dataColumnNames, header=0).set_index('L (meters)')

In [81]:
%%markdown

# Lab {labNum.value}: {labName.value}
Name: **{name.value}** \
Class: **{course.value}** \
Date: **{date.value}**

<p style="page-break-after:always;"></p>


# Lab 2: Simple Pendulum
Name: **Your Name** \
Class: **PHYS 2125 (15921)** \
Date: **2025-01-24**

<p style="page-break-after:always;"></p>


## Objective

To determine the Earth's gravitational constant ($g = 9.8 m/s^2$).

## Equipment

- (1) Small A-base
- (1) long metal rod
- (1) short rod
- (1) roll of string
- (1) calipers
- (1) clamp
- (1) 0.0635 meter aluminum cyliner as pendulum weight
- (1) meter stick
- (1) stop watch

## Theory

The **period of oscilation** of a pendulum ($T$) is the amount of time required to complete a full cycle (*oscilation*). It is related to the **gravitational constant of Earth** ($g$) per the equation:

$$ T = 2\pi \sqrt{\frac{L}{g}} $$

where:

$ T = $ period of oscilation in $ seconds $ \
$ L = $ length of pendulum in $ meters $ \
$ g = $ gravitational constant in $ \frac{meters}{seconds^2} $, i.e $ 9.8 m/s^2 $

The value of $ L $ is the sum of the distance from the pivot point to the weight and half the length of the weight

$ L = l + \frac{1}{2}h $ \
$ l = $ length from the pivot point to the center of the swinging weight in $ meters $ \
$ h = $ height of the cylinder in $ meters $ 

** !! TODO: include diagram/photo of the l and h variables. l is from the top of the weight to the attachment point. h is the height of the cylinder. !! **

** !! TODO: explain how linearizing this relationship in this way helps us to quantify the relationship to work out g.. !! ** ...With this relationship established we can linearize the equation $ T = 2\pi \sqrt{\frac{L}{g}} $, establishing a relationship such that....

$ \begin{align*}
T &= 2\pi \sqrt{\frac{L}{g}} \\
T^2 &= 4\pi^2 - \frac{L}{g} \\
T^2 &= \frac{4\pi^2}{g}L \\
slope &= \frac{4\pi^2}{g} \\
g &= \frac{4\pi^2}{slope}
\end{align*}$

** !! TODO: Add details about how we're going to graph things, and then graph them **
We will graph..... \
$ T^2\;(seconds^2) $ will serve as the independent variable (graphed on the $x$-axis). \
$ L\;(meters) $ will serve as the dependent variable (graphed on the $y$-axis).

## Procedure:

The following procedure was followed:
** !! TODO: Lay out the procedure in exruciating detail **

1. Step 1
1. Step 2
1. Step 5

# Data

** !! TODO: Ignore the CSV column names, use column indexes to access data and/or manually override the headers **

In [82]:
from IPython.display import display, HTML

# !! TODO add  gavg = average of all values of g. show at the bottom of the g column

HTML(df.to_html())

,l (meters),h (meters),5T₁ (seconds),5T₂ (seconds),T₁ (seconds),T₂ (seconds),T (seconds),T² (seconds²),g (meters/seconds²)
L (meters),,,,,,,,,
0.18375,0.152,0.0635,4.55,4.58,0.910,0.916,0.913,0.833569,8.702530
0.23175,0.200,0.0635,5.04,4.90,1.008,0.980,0.994,0.988036,9.259909
0.28375,0.252,0.0635,5.44,5.44,1.088,1.088,1.088,1.183744,9.463196
0.33175,0.300,0.0635,5.87,5.95,1.174,1.190,1.182,1.397124,9.374232
0.38175,0.350,0.0635,6.32,6.35,1.264,1.270,1.267,1.605289,9.388270
0.43175,0.400,0.0635,6.60,6.65,1.320,1.330,1.325,1.755625,9.708683
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.438858


# Calculations

In [86]:
# Calculation

# Calculate the standard deviation of g<avg>
deviation = df.std(ddof=0)
deviation_g = deviation[dataColumnNames[8]];
deviation_g_str = str("%.2f" % round(deviation_g, 2))

# Plot the graph of T^2 (y-axis) vs L (x-axis)
# Show all of the data points
# Label the axes, including units
# Draw the trendline (don't do connectng points). Use least squares method.
# Show the equation of the line on the page of the graph

# use the equation of the line to determine the slope

# Use this slop to determine g as given in the theory.
#
# Calculate the standard deviation of the six values of g in the table.
#
# Calculate percent error of two values of g you calculated. (avg vs graphed)
#



# Results

In [87]:
%%markdown

g-avg = \
g-avg σ = \
g-avg error % = \
g-graph = \
g-graph σ = \
g-graph error % = 

g = g avg +/i % error \
 = g avg +/- StdDev \
 = g graph +/ % error \
Standard Deviation of g<avg> is {deviation_g_str}

## Results

g avg = (and std dev)
g graph = (and std dev)



g-avg = \
g-avg σ = \
g-avg error % = \
g-graph = \
g-graph σ = \
g-graph error % = 

g = g avg +/i % error \
 = g avg +/- StdDev \
 = g graph +/ % error \
Standard Deviation of g<avg> is 0.33

## Results

g avg = (and std dev)
g graph = (and std dev)


In [89]:
%%markdown

## Discussion

* What is the most challenging measurements during data collection and explain why? (4 pts) *

{dq1.value}

* Why do you include half of the height of the cylinder for the length of the pendulum? (3 pts) *
                                                              
{dq2.value}

* What changes of the period of the pendulum you would expect, if you changed the mass of the pendulum? (3 pts) *
    
{dq3.value}


## Discussion

* What is the most challenging measurements during data collection and explain why? (4 pts) *

Your Answer to Question 1

* Why do you include half of the height of the cylinder for the length of the pendulum? (3 pts) *
                                                              
Your Answer to Question 2

* What changes of the period of the pendulum you would expect, if you changed the mass of the pendulum? (3 pts) *
    
Your Answer to Question 3
